#Overview & motivation

All of our team members enjoy movies. In addition to enjoying movies, we also enjoy working with API’s and somewhat structured data sets. Therefore, determining what makes a movie successful using the data available in the Internet Movie Database (IMDB) and Wikipedia seemed like a natural choice.

In [1]:
from imdb import IMDb
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import cPickle as pickle
ia = IMDb(accessSystem='http')
from collections import defaultdict 
import io
from scraper import load_pickled_movies
from datetime import datetime
import time

###Related Work

#Initial Questions

#Data

In [2]:
# Load AAdict (dict of Oscar nominated movies)
AAdict = pickle.load(open('AAdict','rb'))
# Load movies (dict of all movies)
#movies = pickle.load(io.open('moviestemp.p','rb'))

In [6]:
#reload AAdict
#AAdf = pickle.load(open('AAdf','rb'))

#reload AAdf
filename = 'AAdf.p'
AAdf = pickle.load(io.open(filename,'rb'))

In [7]:
AAdf

,Nominated Best Actor,Nominated Best Actress,Nominated Best Animated Feature Film,Nominated Best Art Direction,"Nominated Best Art Direction, Black-and-White","Nominated Best Art Direction, Color",Nominated Best Assistant Director,Nominated Best Cinematography,"Nominated Best Cinematography, Black-and-White","Nominated Best Cinematography, Color",...,Won Special Achievement Award,Won Special Award,country,nominations,title,won,year,movieid,release date,mpaa
0011128,False,False,False,False,False,False,False,False,False,False,...,False,False,0,"[Best Short Subject, One-reel]",Down on the farm,[],1941,0011128,1920-04-18,NaN
0015864,False,False,False,False,False,False,False,False,False,False,...,False,False,0,"[Best Music, Dramatic Picture, Best Sound Reco...",The Gold Rush,[],1942,0015864,1925-06-26,NaN
0017743,False,False,False,False,False,False,False,False,False,False,...,False,False,0,[Best Unique and Artistic Picture],Chang,[],1927,0017743,1927-04-29,NaN
0017781,False,True,False,False,False,False,False,False,False,False,...,False,False,0,[Best Actress],Coquette,[Best Actress],1928,0017781,NaT,NaN
0017806,False,False,False,False,False,False,False,True,False,False,...,False,False,0,[Best Cinematography],The Devil Dancer,[],1927,0017806,1927-11-03,NaN
0017822,False,False,False,True,False,False,False,False,False,False,...,False,False,0,"[Best Art Direction, Best Short Subject, Live ...",The Dove,[Best Art Direction],1927,0017822,1927-12-31,NaN
0017979,False,False,False,False,False,False,False,False,False,False,...,False,False,0,"[Best Writing, Title Writing]",The Private Life of Helen of Troy,[],1927,0017979,1927-12-09,NaN
0018129,False,False,False,False,False,False,False,True,False,False,...,False,False,0,[Best Cinematography],The Magic Flame,[],1927,0018129,1927-09-18,NaN
0018253,True,False,False,False,False,False,False,False,False,False,...,False,False,0,[Best Actor],The Patent Leather Kid,[],1927,0018253,1927-08-15,NaN
0018379,False,True,False,True,False,False,False,False,False,False,...,False,False,0,"[Best Picture, Best Actress, Best Director, Dr...",7th Heaven,"[Best Actress, Best Director, Drama, Best Writ...",1927,0018379,1927-05-06,NaN


In [3]:
# convert AAdict to pandas
AAdf = pd.DataFrame.from_dict(AAdict).transpose()
AAdf['movieid'] = AAdf.index
AAdf.head()
#AAdf[AAdf['Nominated Best Actor']==1].head()

,Nominated Best Actor,Nominated Best Actress,Nominated Best Animated Feature Film,Nominated Best Art Direction,"Nominated Best Art Direction, Black-and-White","Nominated Best Art Direction, Color",Nominated Best Assistant Director,Nominated Best Cinematography,"Nominated Best Cinematography, Black-and-White","Nominated Best Cinematography, Color",...,"Won Best Writing, Title Writing",Won Honorary Award,Won Special Achievement Award,Won Special Award,country,nominations,title,won,year,movieid
0011128,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,0,"[Best Short Subject, One-reel]",Down on the farm,[],1941,0011128
0015864,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,0,"[Best Music, Dramatic Picture, Best Sound Reco...",The Gold Rush,[],1942,0015864
0017743,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,0,[Best Unique and Artistic Picture],Chang,[],1927,0017743
0017781,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,0,[Best Actress],Coquette,[Best Actress],1928,0017781
0017806,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,0,[Best Cinematography],The Devil Dancer,[],1927,0017806


In [4]:
def get_releasedate(row):
    try:
        movobj=ia.get_movie(row['movieid'])
        ia.update(movobj, 'release dates')
        date = str(movobj.data['release dates']).split("USA::", 1)[1]
        day = str(date.split(" ")[0])
        month = str(date.split(" ")[1])
        year = str(date.split(" ")[2].split("'")[0])
        releasedate = datetime.strptime(year + "-" + month + "-" + day, "%Y-%B-%d")
        #releasedate = " ".join(str(movobj.data['release dates']).split("USA::", 1)[1].split(" ")[:3])
    except:
        releasedate = np.nan
    return releasedate

In [5]:
AAdf['release date'] = AAdf.apply(lambda row: get_releasedate(row), axis=1)

In [7]:
def get_mpaa(row):
    try:
        #mpaa =ia.get_movie(row['movieid']).data['mpaa'].split(" ")[0]
        mpaa = str(ia.get_movie(row['movieid']).data['mpaa']).split("Rated ", 1)[1].split(" ")[0]
    except:
        mpaa = np.nan
    return mpaa

In [8]:
AAdf['mpaa'] = AAdf.apply(lambda row: get_mpaa(row), axis=1)

2015-12-01 13:10:25,700 CRITICAL [imdbpy] /Users/Dana/anaconda/lib/python2.7/site-packages/imdb/_exceptions.py:35: IMDbDataAccessError exception raised; args: ({'exception type': 'IOError', 'url': 'http://akas.imdb.com/title/tt0255451/combined', 'errcode': 'socket error', 'proxy': '', 'original exception': IOError('socket error', timeout('timed out',)), 'errmsg': 'timed out'},); kwds: {}
Traceback (most recent call last):
  File "/Users/Dana/anaconda/lib/python2.7/site-packages/imdb/parser/http/__init__.py", line 202, in retrieve_unicode
    uopener = self.open(url)
  File "/Users/Dana/anaconda/lib/python2.7/urllib.py", line 213, in open
    return getattr(self, name)(url)
  File "/Users/Dana/anaconda/lib/python2.7/urllib.py", line 351, in open_http
    errcode, errmsg, headers = h.getreply()
  File "/Users/Dana/anaconda/lib/python2.7/httplib.py", line 1207, in getreply
    response = self._conn.getresponse()
  File "/Users/Dana/anaconda/lib/python2.7/httplib.py", line 1132, in getresp

In [15]:
# To Save new AAdict, run this cell
#filename = 'AAdf.p'
#pickle.dump(AAdf, io.open(filename,'wb'))

In [18]:
# Get list of all unique awards
sum(AAdf['won'])

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [13]:
# Baseline Classifier:
AAdf[AAdf['Nominated Best Actor']==True]['Won Best Actor'].mean()

0.22391857506361323

#Exploratory Data Analysis

#Final Analysis

#Presentation